# load pre-generated  and sampled node text and similar node examples

In [15]:
import json
import random

filename = '../dataset/pubmed_prompt.json'
with open(filename, 'r') as f:
    pubmed = json.load(f)


In [16]:
for key in pubmed["8"]:
    print(key, ":")
    print(pubmed["8"][key])
    print("\n")

id :
8


lable :
Type 2 diabetes


sample_text :
{"id": 8, "title": "Exenatide versus insulin glargine in patients with suboptimally controlled type 2 diabetes: a randomized trial.", "category": "Masked", "author": ["Heine RJ", "Van Gaal LF", "Johns D", "Mihm MJ", "Widel MH", "Brodows RG"], "abstract": "BACKGROUND: Physicians may use either insulin or exenatide injections for patients with type 2 diabetes mellitus who have poor glycemic control despite taking oral blood glucose-lowering drugs. OBJECTIVE: To compare effects of exenatide and insulin glargine on glycemic control in patients with type 2 diabetes mellitus that is suboptimally controlled with metformin and a sulfonylurea. DESIGN: 26-week multicenter, open-label, randomized, controlled trial. SETTING: 82 outpatient study centers in 13 countries. PATIENTS: 551 patients with type 2 diabetes and inadequate glycemic control (defined as hemoglobin A1c level ranging from 7.0% to 10.0%) despite combination metformin and sulfonylurea

# Inference
insert your openai.api_key
or you can revise the get_completion funtion so that you can use other LLMs

In [25]:
import openai
#model="gpt-3.5-turbo-16k" 
#model="gpt-4" 
model="gpt-4-1106-preview"
openai.organization = ""
openai.api_key = ""
s
def get_completion(prompt, model=model):    
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.1, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [26]:
def get_inductive_prompt(similar_examples):
    
    prompt = similar_examples
    prompt += "\n"
    prompt += f"""
        You are a medical expert. For the given example and your own knowledges. 
        Can you find some reasons that why each paper falls into its category. Only briefly list the reasons
        """
    return prompt


def get_deductive_prompt(sample_text, similar_examples, reasons):
    prompt = "Examples: \n"
    prompt += similar_examples
    prompt += "Reasons: \n"
    prompt += reasons
    prompt += "\n"
    prompt += "Target Paper: \n"
    prompt += sample_text + "\n"
    prompt += """
    the "Target Paper" should be in one of which category blow?
    1:'Experimentally induced diabetes',
    2:'Type 1 diabetes',
    3:'Type 2 diabetes',
        
        
    Based on the reasons and examples, think step by step and print reasoning process, 
    then choose the category name.
    output all prediction and reasoning in json format:
    {
        reasoning:, 
        prediction: 
    }
    """
    prompt += '\n'
    return prompt

In [28]:
cnt = 0
tp = 0
verbose = False
data = pubmed

for idx in data:
    print("reasoning on sample:" , idx)
    sample = data[idx]
    similar_examples = sample["similar_examples"]
    
    inductive_prompt = get_inductive_prompt(similar_examples)
    reasons = get_completion(inductive_prompt)
    sample_text = sample["sample_text"]
    deductive_prompt = get_deductive_prompt(sample_text, similar_examples, reasons)
    res = get_completion(deductive_prompt)

    if model == "gpt-3.5-turbo-16k":
        json_string = json.loads(res)
    elif model == "gpt-4-1106-preview":
        start = res.find("json") + len("json")
        end = res.find("```", start)
        json_string = res[start:end].strip()
        res_dict = json.loads(json_string)
    
    if verbose:
        #print("similar_examples: \n", similar_examples, "\n")
        #print("reasons: \n", reasons, "\n")
        print("deductive_prompt: \n",deductive_prompt, "\n")
        print("result: \n", res, "\n")
        print("json_result: \n", res_dict, "\n")
        print("lable:", sample["lable"],"\n")
        print("===========================================")
    
    if res_dict["prediction"] == sample["lable"]:
        tp += 1
    cnt += 1

In [19]:
tp/cnt

1.0